In [1]:
# import statements

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle
from math import comb
import statistics

In [2]:
ibs_graph = pickle.load(open("ibs_graph.pkl", "rb"))
ibd_graph = pickle.load(open("ibd_graph.pkl", "rb"))

classifier = "family"

In [5]:
# CDF (cumulative distribution function)
# calculate Pr(v <= x): probability a node v has <= x degree
# MARKOVS INEQUALITY - MAKE SURE WE INCLUDE IN PAPER

n = len(ibs_graph.nodes)
n2 = len(ibd_graph.nodes)
nm_max = max(n, n2)
expected = (nm_max) / 2

degree_dict = {}

cdf = np.zeros(nm_max)
cdf_ibd = np.zeros(nm_max)
max_deg_ibs = 0
max_deg_ibd = 0

for node in ibs_graph.nodes:
    degree = len(ibs_graph.edges(node))
    cdf[degree] += 1
    if degree > max_deg_ibs:
        max_deg_ibs = degree

for node in ibd_graph.nodes:
    degree = len(ibd_graph.edges(node))
    cdf_ibd[degree] += 1
    if degree > max_deg_ibd:
        max_deg_ibd = degree

max_deg = max(max_deg_ibs, max_deg_ibd)

cdf_ibs_sum = np.zeros(max_deg + 1)
cdf_ibd_sum = np.zeros(max_deg + 1)
markovs = np.zeros(max_deg + 1)

for i in range(max_deg + 1):
    if i > 0:
        cdf_ibs_sum[i] = cdf[i] + cdf_ibs_sum[i-1]
        cdf_ibd_sum[i] = cdf_ibd[i] + cdf_ibd_sum[i-1]

    else:
        cdf_ibs_sum[i] = cdf[i]
        cdf_ibd_sum[i] = cdf_ibd[i]

    if i > 0:
        markovs[i] = 1 - expected / i
    else: 
        markovs[i] = 0
    markovs[i] = max(0, markovs[i])
    markovs[i] = min(1, markovs[i])

degrees = np.arange(0, max_deg + 1, 1)

cdf_ibs_sum = cdf_ibs_sum / n
cdf_ibd_sum = cdf_ibd_sum / n2

# print(degrees)
print(cdf_ibs_sum)
print(cdf_ibd_sum)
# print(cdf_ibd[0])
print(markovs)


[0.76075731 0.81927711 0.84681583 0.86230637 0.87607573 0.88468158
 0.88984509 0.89500861 0.90361446 0.90877797 0.91222031 0.91910499
 0.92598967 0.92943201 0.92943201 0.93459552 0.93803787 0.94664372
 0.95008606 0.95180723 0.95524957 0.95524957 0.96213425 0.96213425
 0.96557659 0.96729776 0.96901893 0.9707401  0.9707401  0.97418244
 0.97418244 0.97934596 0.97934596 0.98106713 0.98795181 0.98795181
 0.98967298 0.99139415 0.99311532 0.99311532 0.99483649 0.99655766
 0.99827883 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ]
[0.66343042 0.69255663 0.7184466  0.74757282 0.75728155 0.76699029
 0.77346278 0.78640777 0.79288026 0.80906149 0.80906149 0.81229773
 0.81553398 0.82524

In [6]:
# plot

plt.figure(figsize = (15, 15), dpi = 600)
plt.plot(degrees, cdf_ibs_sum, label = "IBS")
plt.plot(degrees, cdf_ibd_sum, label = "IBD")
# plt.plot(degrees, markovs, label = "Markov's")
plt.xlabel("degree")
plt.ylabel("Cumulative Data Function (CDF), Pr(v <= x)")
plt.legend()

In [7]:
# number of edges (ibs)

m = len(ibs_graph.edges)
print("vertices:", n, "edges:", m, "average degree:", m / n)

deg_list = []
deg_list_nonzero = []

for node in ibs_graph.nodes:
    deg = len(ibs_graph.edges(node))
    deg_list.append(deg)
    if deg > 0:
        deg_list_nonzero.append(deg)

print("IBS median degree:", statistics.median(deg_list), "median nonzero degree:", statistics.median(deg_list_nonzero))
print("clustering coeff:", nx.average_clustering(ibs_graph))

vertices: 581 edges: 731 average degree: 1.2581755593803787
IBS median degree: 0 median nonzero degree: 5
clustering coeff: 0.13614302071719506


In [8]:
# number of edges (ibd)

m = len(ibd_graph.edges)
print("vertices:", n, "edges:", m, "average degree:", m / n)

deg_list = []
deg_list_nonzero = []

for node in ibd_graph.nodes:
    deg = len(ibd_graph.edges(node))
    deg_list.append(deg)
    if deg > 0:
        deg_list_nonzero.append(deg)

print("IBD median degree:", statistics.median(deg_list), "median nonzero degree:", statistics.median(deg_list_nonzero))
print("clustering coeff:", nx.average_clustering(ibd_graph))

vertices: 581 edges: 1017 average degree: 1.7504302925989672
IBD median degree: 0 median nonzero degree: 16.0
clustering coeff: 0.23473499997595043


In [11]:
# intersection of IBS and IBD

intersection = nx.intersection(ibs_graph, ibd_graph)
union = nx.disjoint_union(ibs_graph, ibd_graph)
plt.figure(figsize=(15,15), dpi = 600)

nx.draw_circular(intersection, node_size = 1, font_size = 8, width = 0.5)

plt.title("Graph of connections between " + classifier)
plt.show()

print("number of edges in intersection:", len(intersection.edges))
print("number of edges in disjoint union:", len(union.edges))

number of edges in intersection: 22
number of edges in disjoint union: 1748
